<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#backtesting-was-a-huge-success" data-toc-modified-id="backtesting-was-a-huge-success-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>backtesting was a huge success</a></span></li></ul></li><li><span><a href="#exchange-APIs" data-toc-modified-id="exchange-APIs-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>exchange APIs</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#.gitignore-note:" data-toc-modified-id=".gitignore-note:-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>.gitignore note:</a></span></li></ul></li><li><span><a href="#oh-right,-a-newly-created-API-key-is-deactivated-for-48-hours-for-security-reasons" data-toc-modified-id="oh-right,-a-newly-created-API-key-is-deactivated-for-48-hours-for-security-reasons-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>oh right, a newly created API key is deactivated for 48 hours for security reasons</a></span></li></ul></li><li><span><a href="#example-authentication-/-transfer-app" data-toc-modified-id="example-authentication-/-transfer-app-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>example authentication / transfer app</a></span><ul class="toc-item"><li><span><a href="#hmac-error" data-toc-modified-id="hmac-error-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>hmac error</a></span></li><li><span><a href="#the-examples-on-coinbasepro's-API-docs-keep-crashing" data-toc-modified-id="the-examples-on-coinbasepro's-API-docs-keep-crashing-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>the examples on coinbasepro's API docs keep crashing</a></span></li><li><span><a href="#let's-use-the-unofficial-wrappers!" data-toc-modified-id="let's-use-the-unofficial-wrappers!-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>let's use the unofficial wrappers!</a></span></li></ul></li><li><span><a href="#wrapper-worked-a-beaut" data-toc-modified-id="wrapper-worked-a-beaut-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>wrapper worked a beaut</a></span><ul class="toc-item"><li><span><a href="#let's-figure-out-how-deposits-work" data-toc-modified-id="let's-figure-out-how-deposits-work-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>let's figure out how deposits work</a></span></li></ul></li></ul></div>

## backtesting was a huge success
1,300 90-day portfolios over 3 years. average investment \$16k

9% returns in 3 months. sounds too good to be true

did i make a laughable calculation error? let's find out by hooking the algo up to an exchange to run with real money

# exchange APIs
probably binance or coinbase. let's run with coinbase for the moment since i already have an account

### .gitignore note:
i'm currently working in a subfolder of the main repo directory. however, i can put 'file_to_ignore.txt' in the main .gitignore, and it doesn't push the file even if it's in a subfolder
- this is good to know when i'm including an API key to access my personal BTC wallet

In [1]:
import secrets # but don't print any key variables to cell output and then push the notebook...

In [2]:
import coinbase

In [17]:
# coinbase = coinbase.with_api_key(str(secrets.api_key), str(secrets.api_secret_key))
# balance = coinbase.get_balance()

# print('Balance is ' + balance + ' BTC')

## oh right, a newly created API key is deactivated for 48 hours for security reasons
makes sense. let's try the coinbasepro docs in the meantime

In [3]:
import cbpro
public_client = cbpro.PublicClient()

https://docs.pro.coinbase.com/#client-libraries

Profiles are the equivalent of portfolios on the Coinbase Pro website. API keys are scoped to a specific profile. An API key only has access to creating and viewing data that belongs to its own profile, unless otherwise noted. This is true for the REST API, FIX API and Websocket Feed. In order to access data or actions on a different profile, please create a new API key via the Coinbase Pro website.

DELETED PROFILES
Profiles can be deleted on the Coinbase Pro website. A deleted profile's API keys' permissions are automatically set to "View".

The REST API has endpoints for account and order management as well as public market data.

REST API ENDPOINT URL
https://api.pro.coinbase.com

Creating a Request
All REST requests must contain the following headers:

CB-ACCESS-KEY The api key as a string.
CB-ACCESS-SIGN The base64-encoded signature (see Signing a Message).
CB-ACCESS-TIMESTAMP A timestamp for your request.
CB-ACCESS-PASSPHRASE The passphrase you specified when creating the API key.
All request bodies should have content type application/json and be valid JSON.



# example authentication / transfer app

 Remember to first base64-decode the alphanumeric secret string (resulting in 64 bytes) before using it as the key for HMAC. Also, base64-encode the digest output before sending in the header.

In [27]:
hkey = base64.b64decode(secrets.pro_secret_key)

In [28]:
hkey

b'{\xc7t?\x1cVN5\x81{\xa9\xee\xf7tK\xd5F\xf1\xe5\x84\xa7R\xfd\n\xeeceJb\x0f\xd9\xd4\xc7\xebH\xda\xfe\xcd?\x9aKR\x8d\x87\x7f\x8a\xadB\x17\xefd:A\\xtK\xd8\xcbW/:2]'

In [ ]:
        timestamp = str(time.time())
        message = (timestamp + request.method + request.path_url +
                   (request.body or ''))
        message = message.encode('ascii')
        hmac_key = base64.b64decode(self.secret_key)
        signature = hmac.new(hmac_key, message, hashlib.sha256)
        signature_b64 = base64.b64encode(signature.digest())

In [60]:
import json, hmac, hashlib, time, requests, base64
from requests.auth import AuthBase

class CoinbaseExchangeAuth(AuthBase):
    
    def __init__(self, api_key, secret_key, passphrase):
        self.api_key = api_key
        self.secret_key = secret_key
        self.passphrase = passphrase
        
    def __call__(self, request): # return request...?
        timestamp = str(time.time())
        message = (timestamp + request.method + request.path_url + (request.body or ''))
        message = message.encode('ascii')
        hmac_key = base64.b64decode(self.secret_key)  # why does this crash?
        signature = hmac.new(hmac_key, message, hashlib.sha256)
#         signature_b64 = signature.digest().encode('base64').rstrip('\n')
        # AttributeError: 'bytes' object has no attribute 'encode'
        signature_b64 = base64.b64encode(signature.digest())
        # oh, it's a syntax thing. hilarious

        request.headers.update({
            'CB-ACCESS-SIGN': signature_b64,
            'CB-ACCESS-TIMESTAMP': timestamp,
            'CB-ACCESS-KEY': self.api_key,
            'CB-ACCESS-PASSPHRASE': self.passphrase,
            'Content-Type': 'application/json'})
        return request

In [61]:
api_url = 'https://api.pro.coinbase.com/'
auth = CoinbaseExchangeAuth(secrets.pro_api_key, secrets.pro_secret_key, secrets.passphrase)

# Get accounts
r = requests.get(api_url + 'accounts', auth=auth)
# print r.json()
type(r)

requests.models.Response

In [62]:
r.content

b'[{"id":"87492cf3-986b-4ad0-93d4-01020c99d85a","currency":"ALGO","balance":"0.0000000000000000","hold":"0.0000000000000000","available":"0","profile_id":"e85c9857-e0f1-4034-985a-e283602d8776","trading_enabled":true},{"id":"e071d6ff-9245-4513-8a9d-a7737d624833","currency":"ATOM","balance":"0.0000000000000000","hold":"0.0000000000000000","available":"0","profile_id":"e85c9857-e0f1-4034-985a-e283602d8776","trading_enabled":true},{"id":"9fb6bc8d-fea0-45d6-bacc-3142a6b52de1","currency":"BAND","balance":"0.0000000000000000","hold":"0.0000000000000000","available":"0","profile_id":"e85c9857-e0f1-4034-985a-e283602d8776","trading_enabled":true},{"id":"e3ea6c25-1127-42da-a6c6-4b8c2e420388","currency":"BAT","balance":"0.0000000000000000","hold":"0.0000000000000000","available":"0","profile_id":"e85c9857-e0f1-4034-985a-e283602d8776","trading_enabled":true},{"id":"d3aa8a1c-58dc-4e93-9338-79575b394e40","currency":"BCH","balance":"0.0000000000000000","hold":"0.0000000000000000","available":"0","prof

## hmac error
seems to be an issue with hmac.new(), particularly the 'message' parameter we pass in
i'm tearing my already thinning hair out over this, but perhaps there's an easier solution (thank you colin for rubberducking me here).
- dan's API wrapper hasn't implemented all the base functionality (i think?) so i'm trying to figure out the base deposit() methods without wrappers
- dan's API wrapper worked just fine to authorize and view my accounts
    - therefore, his python3 wrappers are handling this encoding error the base example is throwing
    - maybe it's as simple as version syntax. either way, i can check his .py files i installed
- AH it worked. brilliant
    - oh, he encodes the message with ascii. huh.


In [ ]:
# dan's wrapper authorization:
class CoinbaseProAuth(AuthBase):

    def __init__(self, api_key, secret_key, passphrase):
        self.api_key = api_key
        self.secret_key = secret_key
        self.passphrase = passphrase

    def __call__(self, request):
        timestamp = str(time.time())
        message = (timestamp + request.method + request.path_url +
                   (request.body or ''))
        message = message.encode('ascii')
        hmac_key = base64.b64decode(self.secret_key)
        signature = hmac.new(hmac_key, message, hashlib.sha256)
        signature_b64 = base64.b64encode(signature.digest())
        request.headers.update({
            'CB-ACCESS-SIGN': signature_b64,
            'CB-ACCESS-TIMESTAMP': timestamp,
            'CB-ACCESS-KEY': self.api_key,
            'CB-ACCESS-PASSPHRASE': self.passphrase,
            'Content-Type': 'application/json'
        })
        return request

In [ ]:
# Place an order
order = {
    'size': 1.0,
    'price': 1.0,
    'side': 'buy',
    'product_id': 'BTC-USD',
}
r = requests.post(api_url + 'orders', json=order, auth=auth)
print r.json()
# {"id": "0428b97b-bec1-429e-a94c-59992926778d"}

https://docs.python.org/3/library/hmac.html

hmac.new(key, msg=None, digestmod='')
Return a new hmac object. key is a bytes or bytearray object giving the secret key. If msg is present, the method call update(msg) is made. digestmod is the digest name, digest constructor or module for the HMAC object to use. It may be any name suitable to hashlib.new(). Despite its argument position, it is required.

Changed in version 3.4: Parameter key can be a bytes or bytearray object. Parameter msg can be of any type supported by hashlib. Parameter digestmod can be the name of a hash algorithm.

Deprecated since version 3.4, removed in version 3.8: MD5 as implicit default digest for digestmod is deprecated. The digestmod parameter is now required. Pass it as a keyword argument to avoid awkwardness when you do not have an initial msg.

hmac.digest(key, msg, digest)
Return digest of msg for given secret key and digest. The function is equivalent to HMAC(key, msg, digest).digest(), but uses an optimized C or inline implementation, which is faster for messages that fit into memory. The parameters key, msg, and digest have the same meaning as in new().

CPython implementation detail, the optimized C implementation is only used when digest is a string and name of a digest algorithm, which is supported by OpenSSL.

New in version 3.7.

In [32]:
# api_url = 'https://api.pro.coinbase.com/'
# auth = CoinbaseExchangeAuth(secrets.pro_api_key, secrets.pro_secret_key, secrets.passphrase)
# # get accounts
# r = requests.get(api_url+'accounts', auth=auth)
# # type(r)

In [ ]:
api_url = 'https://api.pro.coinbase.com/'
auth = CoinbaseExchangeAuth(API_KEY, API_SECRET, API_PASS)

# Get accounts
r = requests.get(api_url + 'accounts', auth=auth)
print r.json()
# [{"id": "a1b2c3d4", "balance":...

# Place an order
order = {
    'size': 1.0,
    'price': 1.0,
    'side': 'buy',
    'product_id': 'BTC-USD',
}
r = requests.post(api_url + 'orders', json=order, auth=auth)
print r.json()
# {"id": "0428b97b-bec1-429e-a94c-59992926778d"}

## the examples on coinbasepro's API docs keep crashing
could be something with python version mismatches. it's written in 2.7. i am constantly vexed by interesting things written in 2.7
## let's use the unofficial wrappers!

In [38]:
import cbpro

key = secrets.pro_api_key
unenc_secret = secrets.pro_secret_key
b64secret = unenc_secret # it does the decoding for us, which eases my headaches around version mismatched syntax
passphrase = secrets.passphrase
auth_client = cbpro.AuthenticatedClient(key, b64secret, passphrase)
# Use the sandbox API (requires a different set of API access credentials)
# auth_client = cbpro.AuthenticatedClient(key, b64secret, passphrase,
#                                   api_url="https://api-public.sandbox.pro.coinbase.com")

In [40]:
accs = auth_client.get_accounts()
# accs

# wrapper worked a beaut
## let's figure out how deposits work
- seems like you need to move funds from your 'main' coinbase accounts to separate 'deposit' portfolios for pro trading

methods are here: https://github.com/danpaquin/coinbasepro-python
ah, but this doesn't seem to manage deposits as i want. maybe (https://github.com/acontry/coinbasepro)?